# 按需分类 KDJ日金叉 & KDJ日死叉 代码仓库

In [5]:
import calculate
import common
from pathlib import Path

# 给原始股票代码加上抬头；'600001' → 'sh600001'
def add_capital(input_file: str, output_file: str):
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            stock_code = line.strip()
            if stock_code.startswith("6"):
                processed_code = f"sh{stock_code}"
            else:
                processed_code = f"sz{stock_code}"
            outfile.write(processed_code + "\n")

    print(f"Processed stock codes saved to {output_file}.")

dict_filename = "./txt_lib/stock_name.txt"
stock_dict = common.Initialization.generate_stock_dict(dict_filename)

## 处理 KDJ日金 + 周死叉 → 弃

In [6]:
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日金.txt"
output_file = "./txt_lib/processed_KDJ日金.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    gold_stock_list = [line.strip() for line in file if line.strip()]

gold_to_del_stock_list = []
gold_to_del_stock_list = common.Final_process.process_stocks(gold_stock_list, mode="week_death")
# print("KDJ日金 因 周死叉 要删除的股票")
# print(f"总计股票数量:{len(gold_to_del_stock_list)}。")

# for stock_code in gold_to_del_stock_list:
#     cur_stock_code = stock_code[2:]
#     stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
#     print(f"{cur_stock_code}:{stock_name}")

Processed stock codes saved to ./txt_lib/processed_KDJ日金.txt.


## 处理 KDJ日死 + 周死叉 → 弃

In [16]:
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日死.txt"
output_file = "./txt_lib/processed_KDJ日死.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    death_stock_list = [line.strip() for line in file if line.strip()]

death_to_del_stock_list = []
death_to_del_stock_list = common.Final_process.process_stocks(death_stock_list, mode="week_death")

# print("KDJ日死 因 周死叉 要删除的股票")
# print(f"总计股票数量:{len(death_to_del_stock_list)}。")

# for stock_code in death_to_del_stock_list:
#     cur_stock_code = stock_code[2:]
#     stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
#     print(f"{cur_stock_code}:{stock_name}")

Processed stock codes saved to ./txt_lib/processed_KDJ日死.txt.


## 处理 KDJ日金 + 日死 → KDJ日死

In [7]:
gold_to_move_day_death_stock_list = []
gold_to_move_day_death_stock_list = common.Final_process.process_stocks(gold_stock_list, mode="daily_death")

# print("KDJ日金 因 日死叉 要转移至 KDJ日死 的股票")
# print(f"总计股票数量:{len(gold_to_move_day_death_stock_list)}。")

# for stock_code in gold_to_move_day_death_stock_list:
#     cur_stock_code = stock_code[2:]
#     stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
#     print(f"{cur_stock_code}:{stock_name}")

## 处理 KDJ日金 + J线拐头 → KDJ8

In [8]:
gold_to_move_8_stock_list = []
gold_to_move_8_stock_list = common.Final_process.process_stocks(
    gold_stock_list, mode="J_turn_around"
)
# print("KDJ日金 因 J线掉头 要转移至 KDJ8 的股票。")
# print(f"总计股票数量:{len(gold_to_move_8_stock_list)}。")

# for stock_code in gold_to_move_8_stock_list:
#     # 不能在 日死叉 list中
#     if stock_code not in gold_to_move_day_death_stock_list:
#         cur_stock_code = stock_code[2:]
#         stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
#         print(f"{cur_stock_code}:{stock_name}")

In [10]:
try:
    folder = Path("./output")
    folder.mkdir()
except Exception as e:
    pass

try:
    folder = Path("./output/KDJ日金")
    folder.mkdir()
    folder = Path("./output/KDJ日死")
    folder.mkdir()
    folder = Path("./output/KDJ8")
    folder.mkdir()
    folder = Path("./output/操作1.1")
    folder.mkdir()
except Exception as e:
    pass

In [ ]:
# 今日金叉数据
today_gold_txt_path = "./output/golden_output/today_gold_cross.txt"
today_gold_list = []
with open(today_gold_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_gold_list.append(stock_code)

# KDJ日金 文件生成
kdj_res = []
for stock_code in gold_stock_list:
    condition1 = stock_code not in gold_to_del_stock_list # 周死
    condition2 = stock_code not in gold_to_move_day_death_stock_list # 日死
    condition3 = stock_code not in gold_to_move_8_stock_list #拐头
    if condition1 and condition2 and condition3:
        kdj_res.append(stock_code)

# 加入今日金叉数据
for stock_code in today_gold_list:
    if stock_code not in kdj_res:
        kdj_res.append(stock_code)

output_file = './output/KDJ日金/KDJ日金.xlsx'
common.Final_file.output_excel(kdj_res,output_file,condition='日金')
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日金/KDJ日金.xlsx
所有图片已生成！


In [ ]:
# KDJ8 文件生成
output_file = "./output/KDJ8/KDJ8.xlsx"
common.Final_file.output_excel(gold_to_move_8_stock_list, output_file, condition="8")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ8/KDJ8.xlsx
所有图片已生成！


In [17]:
# KDJ日死 文件生成
kdj_death = []

# 今日死叉数据
today_death_txt_path = "./output/death_output/today_death_cross.txt"
today_death_list = []
with open(today_death_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_death_list.append(stock_code)

for stock_code in death_stock_list:
    condition1 = stock_code not in death_to_del_stock_list

    if condition1:
        kdj_death.append(stock_code)

# 加入今日死叉数据
for stock_code in today_death_list:
    if stock_code not in kdj_death:
        kdj_death.append(stock_code)

output_file = "./output/KDJ日死/KDJ日死.xlsx"
common.Final_file.output_excel(kdj_death, output_file, condition="日死")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日死/KDJ日死.xlsx
所有图片已生成！
